In [1]:
# Required Libraries

import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import math
import random

2024-05-20 16:09:08.480301: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-20 16:09:08.499061: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-20 16:09:08.499077: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-20 16:09:08.499557: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-20 16:09:08.502865: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
# Transformer Definition (Dependencies)

# Multi-head attention with Q, K, V
class multiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, key_dim, num_heads):
        super(multiHeadAttention, self).__init__()
        self.key_dim = key_dim
        self.num_heads = num_heads

    def build(self, input_shape):
        self.WqL = []
        self.WkL = []
        self.WvL = []

        for i in range(self.num_heads):
            Wq_init = tf.random_normal_initializer()
            Wq = tf.Variable(initial_value=Wq_init(shape=(int(input_shape[-1]), self.key_dim), dtype="float32"), trainable=True)
            self.WqL.append(Wq)

            Wk_init = tf.random_normal_initializer()
            Wk = tf.Variable(initial_value=Wk_init(shape=(int(input_shape[-1]), self.key_dim), dtype="float32"), trainable=True)
            self.WkL.append(Wk)

            Wv_init = tf.random_normal_initializer()
            Wv = tf.Variable(initial_value=Wv_init(shape=(int(input_shape[-1]), int(input_shape[-1])), dtype="float32"), trainable=True)
            self.WvL.append(Wv)

        Wlt_init = init = tf.random_normal_initializer()
        self.Wlt = tf.Variable(initial_value=Wlt_init(shape=((self.num_heads * int(input_shape[-1])), int(input_shape[-1])), dtype="float32"), trainable=True)

    def call(self, inputs):

        # inputs : batch_size x time_steps x dim
        x = inputs

        # transform for generating Q,K,V : (batch_size * time_steps) x dim
        x_tran = tf.reshape(x, [-1])
        x_tran = tf.reshape(x_tran, [-1, int(inputs.shape.as_list()[-1])])

        a_xL = []

        # Generate Query, Key and Value corresponding to each attention head
        for i in range(self.num_heads):

            # Query : batch_size x time_steps x dq
            xq = tf.matmul(x_tran, self.WqL[i])
            xq = tf.reshape(xq, [-1, int(inputs.shape.as_list()[-2]), int(xq.shape.as_list()[-1])])

            # Key : batch_size x time_steps x dk
            xk = tf.matmul(x_tran, self.WkL[i])
            xk = tf.reshape(xk, [-1, int(inputs.shape.as_list()[-2]), int(xk.shape.as_list()[-1])])

            # Value : batch_size x time_steps x dv
            xv = tf.matmul(x_tran, self.WvL[i])
            xv = tf.reshape(xv, [-1, int(inputs.shape.as_list()[-2]), int(xv.shape.as_list()[-1])])

            # Transposing each key in a batch (xk_t : batch_size x dk x time_steps)
            xk_t = tf.transpose(xk, perm=[0, 2, 1])

            # Computing scaled dot product self attention of each time step in each training sample (s_a : batch_size x time_steps x time_steps)
            s_a = tf.math.multiply(tf.keras.layers.Dot(axes=(1, 2))([xk_t, xq]), (1/self.key_dim))

            # Applying Softmax Layer to the self attention weights for proper scaling (sft_s_a : batch_size x time_steps x time_steps)
            sft_s_a = tf.keras.layers.Softmax(axis=2)(s_a)

            # Computing attention augmented values for each time step and each training sample (a_x : batch_size x time_steps x dim)
            a_xL.append(tf.keras.layers.Dot(axes=(1, 2))([xv, sft_s_a]))

        # Concatenate and applying linear transform for making dimensions compatible
        a_x = tf.concat(a_xL, -1)

        # Transform to shape a_x_tran : ((batch_size x time_steps) x (dim x num_heads))
        a_x_tran = tf.reshape(a_x, [-1])
        a_x_tran = tf.reshape(a_x_tran, [-1, (self.num_heads*int(inputs.shape.as_list()[-1]))])

        # Get the dimensions compatible after applying linear transform
        a_x_tran = tf.matmul(a_x_tran, self.Wlt)
        a_x_tran = tf.reshape(a_x_tran, [-1, int(inputs.shape.as_list()[-2]), int(inputs.shape.as_list()[-1])])

        return a_x_tran


# Transformer Block implemented as a Layer
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = multiHeadAttention(embed_dim, num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class PositionEmbeddingLayer(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super(PositionEmbeddingLayer, self).__init__(**kwargs)
        self.position_embedding_layer = layers.Embedding(
            input_dim=(sequence_length), output_dim=output_dim
        )
        self.sequence_length = sequence_length

    def call(self, inputs):
        position_indices = tf.range(self.sequence_length)  #tf.range(1, self.sequence_length + 1, 1)
        embedded_words = inputs
        embedded_indices = self.position_embedding_layer(position_indices)
        return embedded_words + embedded_indices

In [ ]:
# Creating the model
# Initializing the transformer model
def get_transformer_model(num_features, num_attn_heads, hidden_layer_dim, num_transformer_blocks, time_dim):
  transformer_blocks = []

  for i in range(num_transformer_blocks):
      transformer_blocks.append(TransformerBlock(num_features, num_attn_heads, hidden_layer_dim))

  # Model
  inputs = layers.Input(shape=(time_dim, num_features,))
  x = inputs

  # Trainable Embedding
  embedding_layer = PositionEmbeddingLayer(50, num_features)
  x = embedding_layer(x)

  for i in range(num_transformer_blocks):
      x = transformer_blocks[i](x)

  x = layers.GlobalAveragePooling1D()(x)
  x = layers.Dropout(0.2)(x)
  x = layers.Dense(32, activation="relu")(x)
  x = layers.Dropout(0.2)(x)
  outputs = layers.Dense(1)(x)

  model = keras.Model(inputs=inputs, outputs=outputs)

  optim = keras.optimizers.SGD(learning_rate=0.0001)
  model.compile(optimizer=optim, loss='mse', metrics=['mse'])

  return model

# Can use following input arguments
#num_attn_heads = 3
#hidden_layer_dim = 32  # Hidden layer size in feed forward network inside transformer
#num_transformer_blocks = 3
#num_features : Number of features at each time step (in the case of RUL prediction its number of sensors). Last dimension
#               of the dataset
#              Extract Using: num_features = np.asarray(X_tr).astype(np.float32).shape[-1]

#time_dim : History window size (Secondlast dimension of the dataset)
#              Extract Using: time_dim = np.asarray(X_tr).astype(np.float32).shape[-2]

In [ ]:
#Extract Weights Code
for l in range(len(model.weights)):
  weight_l = model.weights[l].numpy()

# Applying Weights Code
for l in range(len(model.weights)):
  model.weights[l].assign(new_weight_l)   #Replace new weight l with the target weight

# Here l is not necessarily the layer, Tensorflow keeps model weights in a list its the index in the list. Some layers like
# transfomers use multiple positions in the list to save weights

In [ ]:
#Client Node

# FL parameters (Set These)
B = None
E = None
num_comm_rounds = None

###Define model
# Can use following input arguments
#num_attn_heads = 3
#hidden_layer_dim = 32  # Hidden layer size in feed forward network inside transformer
#num_transformer_blocks = 3
#num_features : Number of features at each time step (in the case of RUL prediction its number of sensors). Last dimension
#               of the dataset
#              Extract Using: num_features = np.asarray(X_tr).astype(np.float32).shape[-1]

#time_dim : History window size (Secondlast dimension of the dataset)
#              Extract Using: time_dim = np.asarray(X_tr).astype(np.float32).shape[-2]

local_model = get_transformer_model(num_features, num_attn_heads, hidden_layer_dim, num_transformer_blocks, time_dim)

##Put code to load local model data
X_train, Y_train = None, None

###Communication Rounds Loop
for i in range(num_comm_rounds):

  # Get global model weights from Server#
  #----
  # Put code here to get model weights from server (TCP/UPD) and deserialize
  global_model = None
  #----

  # Update Local Model
  for k in range(len(local_model.weights)):
    local_model.weights[k].assign(global_model.weights[k].numpy())

  # Train One Communication Round
  local_model.fit(X_tr, Y_tr, batch_size=B, epochs=E)

  # Send Client Weights to server
  #---
  # Put Code Here (Use local_model.weights to get local model weights as a list)
  #---


In [ ]:
#Server Side

# FL parameters (Set These)
C = None
num_total_clients = None
num_clients_per_round = C * num_total_clients
num_comm_rounds = None

###Define model
# Can use following input arguments
#num_attn_heads = 3
#hidden_layer_dim = 32  # Hidden layer size in feed forward network inside transformer
#num_transformer_blocks = 3
#num_features : Number of features at each time step (in the case of RUL prediction its number of sensors). Last dimension
#               of the dataset
#              Extract Using: num_features = np.asarray(X_tr).astype(np.float32).shape[-1]

#time_dim : History window size (Secondlast dimension of the dataset)
#              Extract Using: time_dim = np.asarray(X_tr).astype(np.float32).shape[-2]

global_model = get_transformer_model(num_features, num_attn_heads, hidden_layer_dim, num_transformer_blocks, time_dim)

###Communication Rounds Loop
for i in range(num_comm_rounds):

  # Get Weights from all Clients
  for j in num_clients_per_round:

    ###Put Code Below
    #---
    # TCP/UPD
    #---

    #Weight Aggregation Rule
    aggregate_weight = None

  for k in range(len(global_model.weights)):
    global_model.weights[k].assign(aggregate_weight.weights[k].numpy())

  # Send Updated Model Weights to Client
  #---
  # Put code here, serialize aggregate_weight and send via TCP/UDP
  #---